In [ ]:
!pip install transformers accelerate safetensors torch torchvision



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [1]:
from transformers import AutoProcessor, LlavaForConditionalGeneration
import torch, gc
from PIL import Image
import requests

In [ ]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(f"Using device: {device}")


Using device: cuda


In [5]:

def recipe(user_img_path, user_input):
    model_id = "llava-hf/llava-1.5-7b-hf"
    model = LlavaForConditionalGeneration.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        low_cpu_mem_usage=True,
    ).to(0)

    processor = AutoProcessor.from_pretrained(model_id)

    torch.cuda.empty_cache()
    gc.collect()


    image = Image.open(user_img_path).convert("RGB")






    conversation = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text":
                "You are a home-style chef. You will be shown a photo of a dish and a creatively described title.\n"
                  "Your task is to generate a simple 2–3 step recipe — just short cooking instructions, using common ingredients.\n"
                  "Keep your answer short and do not list ingredients separately — only include them within the steps.\n"
                  "Your output should be of similar tone, length and format as given in the following examples.\n\n"

                  "### Example 1:\n"
                  "Image: A burger\n"
                  "Dish title: bread stack\n"
                  "Output: Make the patty by mixing beef and bread crumbs. Form into 3/4 inch thick patties and grill them. Assemble the patties, bun, tomatoes, onions, lettuce and sauce. Serve.\n\n"


                  "### Example 2:\n"
                  "Image: A plate of fries\n"
                  "Dish title: golden sticks\n"
                  "Output: Peel the potatoes and cut them into thin sticks. Heat the oil in a frying pan and add the sliced potatoes. Sprinkle with salt. Serve.\n\n"


                  "###Example 3:\n"
                  "Image: donuts\n"
                  "Dish title: sweet ring things\n"
                  "Output: To make the dough, whisk egg, butter, sugar, baking powder, flour in a bowl. Cut out the donuts from the dough using a donut cutter. Fry them for 2–3 minutes. Cover them with sugar, icing, chocolate sauce and sprinkles.\n\n"


                  "Now here’s a new one:"
                },
                {"type": "image"},
                {"type": "text", "text": f"Dish title: {user_input}. What's the recipe?"}
            ],
        },
    ]


    prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)


    inputs = processor(images=image, text=prompt, return_tensors='pt').to(0, torch.float16)



    with torch.no_grad():
            output = model.generate(**inputs, max_new_tokens=100)

    torch.cuda.empty_cache()
    gc.collect()


    response = processor.batch_decode(output[:, inputs["input_ids"].shape[-1]:], skip_special_tokens=True)[0]
    return(response)

In [3]:
import json

In [6]:
with open("/content/VLMdata/data.json", "r") as f:
  all_data = json.load(f)

results = []
for dish in all_data:
  user_img_path = dish["image"]
  user_input = dish["title"]
  print(f"\nDish title (vague): {user_input}")
  recipe_text = recipe(user_img_path, user_input)
  print(f"\nRecipe: {recipe_text}")

  results.append({
        "image_path": user_img_path,
        "dish_title": user_input,
        "generated_recipe": recipe_text
    })

with open("output.json", "w") as f:
    json.dump(results, f)



Dish title (vague): noodly thing


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]


Recipe: Noodly thing.

1. Cook the noodles according to package instructions.
2. In a separate pan, sauté the vegetables (such as peppers, onions, and carrots) until they are tender.
3. Combine the cooked noodles and sautéed vegetables in a bowl.
4. Add your choice of protein (such as chicken, beef, or tofu) and sauce.

Dish title (vague): saucy slice


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]


Recipe: Make the sauce by mixing tomato sauce, garlic, onion, and herbs. Preheat the oven to 400°F. Slice the pizza into 6 equal pieces. Assemble the pizza by spreading the sauce evenly on the pizza dough, adding cheese, and topping with the sliced pizza. Bake for 12-15 minutes or until the crust is golden brown and

Dish title (vague): twirly bites


...


Recipe: Make the pancakes by mixing flour, sugar, baking powder, milk and butter. Cook the pancakes on a hot griddle until bubbly. Assemble the stacks by adding butter and syrup. Serve.

Dish title (vague): food cylinder


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]


Recipe: Make the filling by mixing chicken, rice, and spices. Roll out the tortilla and place the filling in the center. Wrap the tortilla around the filling and cut into slices. Serve with your favorite dipping sauce.

Dish title (vague): mash soft thing


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]


Recipe: Mash soft thing.


In [7]:
print(results)

[{'image_path': '/content/VLMdata/noodle.jpeg', 'dish_title': 'noodly thing', 'generated_recipe': 'Noodly thing.\n\n1. Cook the noodles according to package instructions.\n2. In a separate pan, sauté the vegetables (such as peppers, onions, and carrots) until they are tender.\n3. Combine the cooked noodles and sautéed vegetables in a bowl.\n4. Add your choice of protein (such as chicken, beef, or tofu) and sauce.'}, {'image_path': '/content/VLMdata/pizza.jpg', 'dish_title': 'saucy slice', 'generated_recipe': 'Make the sauce by mixing tomato sauce, garlic, onion, and herbs. Preheat the oven to 400°F. Slice the pizza into 6 equal pieces. Assemble the pizza by spreading the sauce evenly on the pizza dough, adding cheese, and topping with the sliced pizza. Bake for 12-15 minutes or until the crust is golden brown and'}, {'image_path': '/content/VLMdata/pasta.jpg', 'dish_title': 'twirly bites', 'generated_recipe': 'Make the pasta by boiling the noodles according to the package instructions.

In [8]:
from google.colab import files

files.download("output.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>